In [1]:
from pulp import *
import pandas as pd
import numpy as np

In [2]:
##sample data.  all dictionaries for better indexing
##this would ideally be in csv format and would be indexed, changed to matrix (numpy) and added into the 
##multi-commodity model
products=["first_product","second_product","third_product"]

ORIG= ["GARY",
       "CLEV",
       "PITT",
       "Order_More"]

##supply at each origin
supply_1=np.array([400,700,800])
supply_2=np.array([8000,1600,1800])
supply_3=np.array([200,300,3000])


supply=[]
supply.append(supply_1)
supply.append(supply_2)
supply.append(supply_3)

        
DEST=["FRA",
      "DET",
      "LAN",
      "WIN",
      "STL",
      "FRE",
      "LAF",
      "Inventory"]

##demand at each destination
demand_1=np.array([4000,300,100,75,650,225,250])
demand_2=np.array([500,750,400,250,950,850,500])
demand_3=np.array([100,100,0,50,200,100,250])

demand=[]
demand.append(demand_1)
demand.append(demand_2)
demand.append(demand_3)

##demand sums
sum_demand=[]
for i in range(0,len(demand)):
    sum_demand.append(sum(demand[i]))

##supply sums
sum_supply=[]
for i in range(0,len(supply)):
    sum_supply.append(sum(supply[i]))
    
##indexing which is greatest accordingly.  taking care of the dummy variable
dummy_sup=[]
dummy_dem=[]

for i in range(0,len(supply)):
    test_supply=[supply[i][x] for x in range(0,len(supply[i]))]
    test_demand=[demand[i][x] for x in range(0,len(demand[i]))]
    test_demand=[demand[i][x] for x in range(0,len(demand[i]))]
    ##check to determine if supply or demand is greater.  each possibility is thought of and taken care
    if sum_demand[i]>sum_supply[i]:
        dummy_sup.append(np.append(test_supply,sum_demand[i]-sum_supply[i]))
        dummy_dem.append(np.append(test_demand,0))
    elif sum_demand[i]<sum_supply[i]:
        dummy_dem.append(np.append(test_demand,sum_supply[i]-sum_demand[i]))
        dummy_sup.append(np.append(test_supply,0))
    elif sum_demand[i]==sum_supply[i]:
        dummy_dem.append(np.append(test_demand,0))
        dummy_sup.append(np.append(test_supply,0))
        
supply=dummy_sup
demand=dummy_dem
demand=np.array(demand)
supply=np.array(supply)

##dictionaries so that the names can be used in the output
##dictionary for demand
DEMAND={}
for p in range(0,len(products)):
    for d in range(0,len(DEST)):
        DEMAND[(products[p],DEST[d])]=demand[p,d]

##dictionary for supply
SUPPLY={}
for p in range(0,len(products)):
    for o in range(0,len(ORIG)):
        SUPPLY[(products[p],ORIG[o])]=supply[p,o]

##time matrix.  cost or money.  depends on what is attempting to be minimized
ship_from_1=np.array([[39,14,11,14,16,82,8,0],
                      [27,9,12,9,26,95,17,0],
                      [24,14,17,13,28,99,20,0],
                       np.zeros(len(DEST))])

ship_from_2=np.array([[39,14,11,14,16,82,8,0],
                      [27,9,12,9,26,95,17,0],
                      [24,14,17,13,28,99,20,0],
                       np.zeros(len(DEST))])

ship_from_3=np.array([[39,14,11,14,16,82,8,0],
                      [27,9,12,9,26,95,17,0],
                      [24,14,17,13,28,99,20,0],
                       np.zeros(len(DEST))])

##able to use a shipping limit to for each hub if needed
ship_cost=[]
ship_cost.append(ship_from_1)
ship_cost.append(ship_from_2)
ship_cost.append(ship_from_3)
ship_cost=np.array(ship_cost)

##constructing a dictionary to keep costs of transporting from one 
##origin to another destination
Cost={}
for p in range(0,len(products)):
    for o in range(0,len(ORIG)):
        for d in range(0,len(DEST)):
            Cost[(products[p],ORIG[o],DEST[d])]=ship_cost[p,o,d]

In [3]:
##defined decision variable
##amounts to be transported from origin points to destination points
Trans = pulp.LpVariable.dicts('Units_to_be_shipped_Origin_destination', [(p,o,d) for p in products for o in ORIG for d in DEST],
                            lowBound =0,
                            cat = 'Integer')


In [4]:
##objective function
model = LpProblem("Supply_and_Demand", LpMinimize)
model += lpSum([Cost[(p,o,d)]*Trans[(p,o,d)] for p in products for o in ORIG for d in DEST])

In [5]:
##could be conditional here on case by case basis.
for p in products:
    for o in ORIG:
        model += lpSum([Trans[p,o,d] for d in DEST]) == SUPPLY[p,o] 

##must have more than or equal to demand.    
for p in products:
    for d in DEST:
        model += lpSum([Trans[p,o,d] for o in ORIG])== DEMAND[p,d]

In [6]:
model.solve()

1

In [7]:
##optimal solution found?
print("Status:", LpStatus[model.status], "\n")

Status: Optimal 



In [8]:
for v in model.variables():
    if v.varValue>0:
        print(v.name,"=", v.varValue)  

Units_to_be_shipped_Origin_destination_('first_product',_'CLEV',_'DET') = 300.0
Units_to_be_shipped_Origin_destination_('first_product',_'CLEV',_'LAF') = 225.0
Units_to_be_shipped_Origin_destination_('first_product',_'CLEV',_'LAN') = 100.0
Units_to_be_shipped_Origin_destination_('first_product',_'CLEV',_'WIN') = 75.0
Units_to_be_shipped_Origin_destination_('first_product',_'GARY',_'LAF') = 25.0
Units_to_be_shipped_Origin_destination_('first_product',_'GARY',_'STL') = 375.0
Units_to_be_shipped_Origin_destination_('first_product',_'Order_More',_'FRA') = 3200.0
Units_to_be_shipped_Origin_destination_('first_product',_'Order_More',_'FRE') = 225.0
Units_to_be_shipped_Origin_destination_('first_product',_'Order_More',_'STL') = 275.0
Units_to_be_shipped_Origin_destination_('first_product',_'PITT',_'FRA') = 800.0
Units_to_be_shipped_Origin_destination_('second_product',_'CLEV',_'DET') = 750.0
Units_to_be_shipped_Origin_destination_('second_product',_'CLEV',_'Inventory') = 600.0
Units_to_be_shi

In [9]:
value(model.objective) 

169000.0

In [10]:
##can be put into a csv or ran and manually input by a tech or 
##client staff

##creating arbitrary csv for each separate product.  can be fine-tuned.
results_dataframes=[]

for d in DEST:
    results_dataframes.append(pd.DataFrame())
    
##need a way to visualize results.  at least have a separate csv for each destination
for d in range(0,len(DEST)):
    ##indices where d is in than for each origin
    results_dataframes[d]['Origin']=ORIG
    for p in products:
        insert_=[]
        for o in ORIG:  
            for v in model.variables():
                if p in v.name and DEST[d] in v.name and o in v.name:
                    insert_.append(v.varValue)
        results_dataframes[d][p+"_to_"+DEST[d]]=insert_

In [11]:
results_dataframes[0]

,Origin,first_product_to_FRA,second_product_to_FRA,third_product_to_FRA
0,GARY,0.0,0.0,0.0
1,CLEV,0.0,0.0,0.0
2,PITT,800.0,500.0,100.0
3,Order_More,3200.0,0.0,0.0


In [12]:
for d in range(0,len(DEST)):
    results_dataframes[d].to_csv(DEST[d]+"_Orders_for_Products.csv")